In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import wget
import pickle as pkl
import sklearn.calibration as calib
from scipy.special import softmax
import sklearn.calibration as calib

In [ ]:
#Load file and assign values to dictionaries,
fi = np.load("ypred.npz")
list(fi.keys())
ygen = fi["ygen"]
ycand = fi["ycand"]
ypred = fi["ypred"]
ypred_raw = fi["ypred_raw"]
X=fi["X"]

In [ ]:
#Concatate all matricies
conc_ygen = np.concatenate(ygen[0:ygen.shape[0]])
conc_ypred= np.concatenate(ypred[0:ygen.shape[0]])
#extract ids
ygen_ids=conc_ygen[:,0]
ypred_ids=conc_ypred[:,0]

In [ ]:
#particle ID (type is)
#0 - no particle
#1 - charged hadron
#2 - neutral hadron
#3 - photon
#4 - electron
#5 - muon
print(np.unique(ygen_ids, return_counts=True))
print(ygen_ids.shape)

In [ ]:
#Create binary problems

#Particle(1) vs no particle(0)
ygen_ids_b=np.where(ygen_ids>0,1,ygen_ids)
ypred_ids_b=np.where(ypred_ids>0,1,ypred_ids)
print(np.unique(ygen_ids, return_counts=True))

#Charged hadron(1) vs else(0)
ygen_ids_CHb=np.where(ygen_ids != 1,0,ygen_ids)
print(np.unique(ygen_ids_CHb, return_counts=True))

#Neutral hadron(1) vs else(0)
ygen_ids_NHb=np.where(ygen_ids != 2,0,ygen_ids)
ygen_ids_NHb=np.where(ygen_ids_NHb == 2,1,ygen_ids_NHb)
print(np.unique(ygen_ids_NHb, return_counts=True))

In [ ]:
#Concatntate and change to binary the raw prediction values using softmax function.
ypred_raw_conc=np.concatenate(ypred_raw[0:ypred.shape[0]])
ypred_sm=softmax(ypred_raw_conc,axis=1)

In [ ]:
#converting raw probabilities into binary. y_pred: Particle(1) Vs NO Particle(0) "ypred_PVNP"
n_events=len(ygen_ids)
ypred_PVNP=[]
for i in range(n_events):
    n_bin=[]
    ypred_NP_prob=ypred_sm[i][0] #probability of no particle
    n_bin.append(ypred_NP_prob) 
    n_bin.append(1-ypred_NP_prob)
    #print(n_bin)
    ypred_PVNP.append(n_bin)
#convert to array
ypred_PVNP=np.asarray(ypred_PVNP)

In [ ]:
#ypred_CH=ypred_sm[1] #Charged hadron probability
ypred_CH=[]
for i in range(n_events):
    n_bin=[]
    CH_prob = ypred_sm[i][1]
    n_bin.append(1-CH_prob)
    n_bin.append(CH_prob)
    #print(n_bin)
    ypred_CH.append(n_bin)
#convert to array
ypred_CH=np.asarray(ypred_CH)

In [ ]:
#ypred_NH=ypred_sm[2] #Neutralhadron
ypred_NH=[]
for i in range(n_events):
    n_bin=[]
    NH_prob = ypred_sm[i][2]
    n_bin.append(1-NH_prob)
    n_bin.append(NH_prob)
    #print(n_bin)
    ypred_NH.append(n_bin)
#convert to array
ypred_NH=np.asarray(ypred_NH)

In [ ]:
#separate probability that an particle is detected form conf
ypred_1=ypred_PVNP[:,1]
#separate probability that an Charged Hadron is detected form conf
ypred_CH1=ypred_CH[:,1]
#separate probability that an Neutral Hadron is detected form conf
ypred_NH1=ypred_NH[:,1]

In [ ]:
#calibration curve sklearn
fraction_of_positives, mean_predicted_value=calib.calibration_curve(ygen_ids_b,ypred_1,normalize=True,n_bins=10,strategy="uniform")
#plotting https://scikit-learn.org/stable/auto_examples/calibration/plot_compare_calibration.html#sphx-glr-auto-examples-calibration-plot-compare-calibration-py

plt.figure(figsize=(10, 10))
ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
ax2 = plt.subplot2grid((3, 1), (2, 0))

ax1.plot(mean_predicted_value, fraction_of_positives, "s-",)

bins = np.linspace(0,1,11)
ax2.hist(ypred_PVNP[ygen_ids_b==0,1],density=False,log=True,bins=bins,color = "red",alpha=0.5,label="ygen_ids=0")
ax2.hist(ypred_PVNP[ygen_ids_b==1,1],density=False,log=True,bins=bins,color = "skyblue",alpha=0.7,label="ygen_ids=1")

ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
ax1.set_ylabel("Fraction of positives")
ax1.set_ylim([-0.05, 1.05])
ax1.legend(loc="lower right")
ax1.set_title('Calibration plots(reliability curve) for particle vs no particle')

ax2.set_xlabel("Mean predicted value")
ax2.set_ylabel("Count")
ax2.legend(loc="upper center", ncol=2)

plt.tight_layout()
#plt.savefig('Particle_vs_no.png')
plt.show()

In [ ]:
#calibration curve sklearn for Charged hadron vs else
import sklearn.calibration as calib
fraction_of_positives, mean_predicted_value=calib.calibration_curve(ygen_ids_CHb,ypred_CH1,normalize=True,n_bins=10,strategy="uniform")
#plotting https://scikit-learn.org/stable/auto_examples/calibration/plot_compare_calibration.html#sphx-glr-auto-examples-calibration-plot-compare-calibration-py

plt.figure(figsize=(10, 10))
ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
ax2 = plt.subplot2grid((3, 1), (2, 0))

ax1.plot(mean_predicted_value, fraction_of_positives, "s-",)


bins = np.linspace(0,1,11)
ax2.hist(ypred_CH[ygen_ids_CHb==0,1],density=False,log=True,bins=bins,color = "red",alpha=0.5,label="ygen_ids=0")
ax2.hist(ypred_CH[ygen_ids_CHb==1,1],density=False,log=True,bins=bins,color = "skyblue",alpha=0.7,label="ygen_ids=1")

ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
ax1.set_ylabel("Fraction of positives")
ax1.set_ylim([-0.05, 1.05])
ax1.legend(loc="lower right")
ax1.set_title('Calibration plots(reliability curve) for Charged Hadron vs else')

ax2.set_xlabel("Mean predicted value")
ax2.set_ylabel("Count")
ax2.legend(loc="upper center", ncol=2)

plt.tight_layout()
#plt.savefig('CH_vs_else2.png')
plt.show()

In [ ]:
#calibration curve sklearn for Neutral hadron vs else
import sklearn.calibration as calib
fraction_of_positives, mean_predicted_value=calib.calibration_curve(ygen_ids_NHb,ypred_NH1,normalize=True,n_bins=10,strategy="uniform")
#plotting https://scikit-learn.org/stable/auto_examples/calibration/plot_compare_calibration.html#sphx-glr-auto-examples-calibration-plot-compare-calibration-py

plt.figure(figsize=(10, 10))
ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
ax2 = plt.subplot2grid((3, 1), (2, 0))

ax1.plot(mean_predicted_value, fraction_of_positives, "s-",)

bins = np.linspace(0,1,11)
ax2.hist(ypred_NH[ygen_ids_NHb==0,1],density=False,log=True,bins=bins,color = "red",alpha=0.5,label="ygen_ids=0")
ax2.hist(ypred_NH[ygen_ids_NHb==1,1],density=False,log=True,bins=bins,color = "skyblue",alpha=0.7,label="ygen_ids=1")


ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
ax1.set_ylabel("Fraction of positives")
ax1.set_ylim([-0.05, 1.05])
ax1.legend(loc="lower right")
ax1.set_title('Calibration plots(reliability curve) for Neutral Hadron vs else')

ax2.set_xlabel("Mean predicted value")
ax2.set_ylabel("Count")
ax2.legend(loc="lower right", ncol=2)

plt.tight_layout()
#plt.savefig('NH_vs_else2.png')
plt.show()